### LIMPIEZA Y TRANSFORMACIÓN DE DATOS

In [324]:
# Cargar librerías
import pandas as pd  # Manejo de dataframes
import sqlite3 as sql  # Manejo de base de datos
from sklearn.preprocessing import LabelEncoder  # Codificar variables categóricas

In [325]:
# Cargar data
df_emp = pd.read_csv("data/employee_survey_data.csv")
df_gen = pd.read_csv("data/general_data.csv", sep=";")
df_in_time = pd.read_csv("data/in_time.csv")
df_manager = pd.read_csv("data/manager_survey_data.csv")
df_out_time = pd.read_csv("data/out_time.csv")
df_retirement = pd.read_csv("data/retirement_info.csv", sep=";")

In [326]:
# Renombrar columnas
df_in_time = df_in_time.rename(columns={"Unnamed: 0": "EmployeeID"})
df_out_time = df_out_time.rename(columns={"Unnamed: 0": "EmployeeID"})

# Establecer índices
df_emp = df_emp.set_index("EmployeeID")
df_gen = df_gen.set_index("EmployeeID")
df_in_time = df_in_time.set_index("EmployeeID")
df_manager = df_manager.set_index("EmployeeID")
df_out_time = df_out_time.set_index("EmployeeID")
df_retirement = df_retirement.set_index("EmployeeID")

# Convertir a formato fecha
df_in_time = df_in_time.apply(pd.to_datetime)
df_out_time = df_out_time.apply(pd.to_datetime)

df_retirement["retirementDate"] = pd.to_datetime(
    df_retirement["retirementDate"], format="%d/%m/%Y"
)

# Convertir columnas a formato float
df_manager["JobInvolvement"] = df_manager["JobInvolvement"].astype(float)
df_manager["PerformanceRating"] = df_manager["PerformanceRating"].astype(float)

# Eliminar columnas innecesarias ya que son redundantes
df_gen = df_gen.drop(["Over18", "StandardHours", "EmployeeCount"], axis=1)

In [322]:
# Ciclo para conocer los nulos en cada dataframe
for df in [df_emp, df_gen, df_manager, df_retirement]:
    print(df.isnull().sum())
    print("\n")

EnvironmentSatisfaction    25
JobSatisfaction            20
WorkLifeBalance            38
dtype: int64


Age                         0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
PercentSalaryHike           0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int64


JobInvolvement       0
PerformanceRating    0
dtype: int64


Attrition             0
retirementDate        0
retirementType        0
resignationReason    70
dtype: int64




In [327]:
#### Nulos en columnas de los dataframes
# Los nulos de la columna NumCompaniesWorked se reemplazan por 0
df_gen["NumCompaniesWorked"] = df_gen["NumCompaniesWorked"].fillna(0)
# Los nulos de la columna resignationReason se reemplazan por "Fired"
df_retirement["resignationReason"] = df_retirement["resignationReason"].fillna("Fired")

# Reemplazar datos nulos de EnvironmentSatisfaction, JobSatisfaction, WorkLifeBalance por la media de ese empleado
df_emp["EnvironmentSatisfaction"] = df_emp["EnvironmentSatisfaction"].fillna(
    df_emp["EnvironmentSatisfaction"].mean()
)
df_emp["JobSatisfaction"] = df_emp["JobSatisfaction"].fillna(
    df_emp["JobSatisfaction"].mean()
)
df_emp["WorkLifeBalance"] = df_emp["WorkLifeBalance"].fillna(
    df_emp["WorkLifeBalance"].mean()
)

# Ciclo para conocer los nulos en cada dataframe
for df in [df_emp, df_gen, df_manager, df_retirement]:
    print(df.isnull().sum())
    print("\n")

EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
dtype: int64


Age                        0
BusinessTravel             0
Department                 0
DistanceFromHome           0
Education                  0
EducationField             0
Gender                     0
JobLevel                   0
JobRole                    0
MaritalStatus              0
MonthlyIncome              0
NumCompaniesWorked         0
PercentSalaryHike          0
StockOptionLevel           0
TotalWorkingYears          9
TrainingTimesLastYear      0
YearsAtCompany             0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
dtype: int64


JobInvolvement       0
PerformanceRating    0
dtype: int64


Attrition            0
retirementDate       0
retirementType       0
resignationReason    0
dtype: int64




In [328]:
# Unir todos los dataframes con el índice con merge
df = (
    df_gen.merge(df_emp, on="EmployeeID", how="outer")
    .merge(df_manager, on="EmployeeID", how="outer")
    .merge(df_retirement, on="EmployeeID", how="outer")
)


### Renombrar Variable objetivo
df = df.rename(columns={"Attrition": "target"})


#### Nulos después de unir los dataframes

# Se eliminan los registros nulos de df_gen["TotalWorkingYears"], porque no se encuentran patrones (9)
df = df.dropna(subset=["TotalWorkingYears"])
# Los nan de la columna "Attrition" se reemplazan por "No"
df["target"] = df["target"].fillna("No")
# Los nan de retirementDate se reemplazan por "No"
# df["retirementDate"] = df["retirementDate"].fillna("No")
# Los nan de retirementType se reemplazan por "No"
df["retirementType"] = df["retirementType"].fillna("No")
# Los nan de resignationReason se reemplazan por "No"
df["resignationReason"] = df["resignationReason"].fillna("No")

### Comprobación de unión exitosa
df.shape

(4401, 28)

In [329]:
# Transformar variable objetivo a binaria
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
df["target"] = enc.fit_transform(df["target"])

In [331]:
# Guardar dataframe en csv para usarlo en el siguiente notebook
df.to_csv("data/df.csv", index=True)